# Introducing RNNs and LSTMs

In [1]:
import autograd
import autograd.misc.optimizers as optim
import autograd.numpy as np
from autograd import grad

import matplotlib.pyplot as plt
%matplotlib inline

## Resources

You may find the following resources helpful for understanding how RNNs and LSTMs work:

* [The Unreasonable Effectiveness of RNNs (Andrej Karpathy)](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
* [Recurrent Neural Networks Tutorial (Wild ML)](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/)
* [Understanding LSTM Networks (Chris Olah)](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

## Character-Level Language Model

In this tutorial, we will reproduce Andrej Karpathy's character level language model. Given a specific character, we use a RNN/LSTM to learn the distribution of the next character. We apply the character level language model on a dataset of Shakespearian text, located at: https://github.com/karpathy/char-rnn/blob/master/data/tinyshakespeare/input.txt

The tutorial is broken into the following sections:
- Part 1: Implemention of RNNs from scratch and application on the dataset.
- Part 2: Implemention LSTMs from scratch and application on the dataset.
- Part 3: Implementation of RNN in PyTorch, and example of batch training using sequential data.
- Part 4: Strategies for batch training irregular length time series.

In [2]:
# Load the Shakespeare text. See above for download link.
with open('./data/shakespeare.txt', 'r') as f:
    text = f.read()

print("------------------------------")
# Print a sample of the text
print(text[:200])
data_length = len(text)
vocab = list(set(text))
vocab_size = len(vocab)   # + 1      # The extra + 1 is for the end-of-string token

char_to_index = { char:index for (index,char) in enumerate(vocab) }
index_to_char = { index:char for (index,char) in enumerate(vocab) }

print("The vocabulary contains {}".format(vocab))
print("------------------------------")
print("TOTAL NUM CHARACTERS = {}".format(data_length))
print("NUM UNIQUE CHARACTERS = {}".format(vocab_size))
print('char_to_index {}'.format(char_to_index))

------------------------------
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you
The vocabulary contains ['m', 'T', 'Y', 'w', 'E', 'Q', 'f', '!', 'I', '$', 's', ';', 'g', 'K', 'y', 'B', ':', 'N', 'q', 'x', 'l', 'L', 'i', 'X', "'", 'u', 'd', 't', 'k', 'v', 'M', 'p', 'J', 'e', 'c', '.', 'o', 'W', 'n', 'b', ',', 'O', 'j', 'Z', '-', 'P', 'S', 'G', 'A', 'a', 'H', 'D', 'C', ' ', 'h', '3', 'U', 'F', 'V', '\n', '&', 'r', 'R', '?', 'z']
------------------------------
TOTAL NUM CHARACTERS = 1115394
NUM UNIQUE CHARACTERS = 65
char_to_index {'m': 0, 'T': 1, 'Y': 2, 'w': 3, 'E': 4, 'Q': 5, 'f': 6, '!': 7, 'I': 8, '$': 9, 's': 10, ';': 11, 'g': 12, 'K': 13, 'y': 14, 'B': 15, ':': 16, 'N': 17, 'q': 18, 'x': 19, 'l': 20, 'L': 21, 'i': 22, 'X': 23, "'": 24, 'u': 25, 'd': 26, 't': 27, 'k': 28, 'v': 29, 'M': 30, 'p': 31, 'J': 32, 'e': 33, 'c': 34, '.': 35,

## Part 1: RNNs

![Recurrent Neural Network Diagram](http://www.wildml.com/wp-content/uploads/2015/09/rnn.jpg)
(Image from the [Wild ML RNN Tutorial](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/))

The update of an RNN is expressed by the following formulas:

$$
h_t = \tanh(U x_t + W h_{t-1} + b_h)
$$

$$
y_t = \text{softmax}(V h_t + b_y)
$$

Here, each $x_t$ is a _character_---in this example, there are 65 unique characters. Since in each step the model takes as input a character and outputs a prediction for the next character in the sequence, both $x_t$ and $o_t$ are 65-dimensional vectors, i.e., $x_t, o_t \in \mathbb{R}^{65}$. We can choose any dimension for the hidden state $h_t$; in this case, we will use $h_t \in \mathbb{R}^{100}$. With this setup, the dimensions of $U$, $W$, and $V$ are $100 \times 65$, $100 \times 100$, and $65 \times 100$, respectively.

Note that we must compute a softmax to obtain the RNN output. For a vector $\mathbf{x}$, we have:

$$
\text{softmax}(\mathbf{x})_i = \frac{e^{\mathbf{x}_i}}{\sum_j e^{\mathbf{x}_j}}
$$

Below, we show a simple trick to improve numerical stability of the softmax.  
Afterwards, we define the RNN model and its initializations.

In [3]:
# Warning: not numerically stable
def softmax_unstable(x):
    return np.exp(x) / np.sum(np.exp(x))

In [4]:
softmax_unstable([1, 2, 1000])

/pkgs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


array([ 0.,  0., nan])

In [5]:
# Numerically stable version
def softmax(x):
    exponential = np.exp(x - np.max(x))
    return exponential / np.sum(exponential)

In [6]:
softmax([1,2,1000])

array([0., 0., 1.])

In [7]:
def log_softmax(x):
    return np.log(softmax(x) + 1e-6)

In [8]:
log_softmax([1,2,1000])

array([-1.38155106e+01, -1.38155106e+01,  9.99999500e-07])

In [9]:
def initialize_params(input_size, hidden_size, output_size):
    params = {
        'U': np.random.randn(hidden_size, input_size) * 0.01,
        'W': np.random.randn(hidden_size, hidden_size) * 0.01,
        'V': np.random.randn(output_size, hidden_size) * 0.01,
        'b_h': np.zeros(hidden_size),
        'b_o': np.zeros(output_size)
    }
    return params

In [10]:
def initialize_hidden(hidden_size):
    return np.zeros(hidden_size)

In [11]:
def model(params, x, h_prev):
    h = np.tanh(np.dot(params['U'], x) + np.dot(params['W'], h_prev) + params['b_h'])
    y = softmax(np.dot(params['V'], h) + params['b_o'])
    return y, h

In [12]:
def criterion(output, target):
    """Negative log-likelihood loss. Useful for training a classification problem with n classes.
    """
    output = np.log(output)
    return -output[target]

In [13]:
def loss(params, input_seq, target_seq, opts):
    """
    Compute the loss of RNN based on data.
    
    :param params: dict of str: tensor, including keys U, W, v, b_h, b_o.
    :param input_seq: list of str. Input string.
    :param target_seq: list of str. Target string.
    :param opts: dict of str: int. Including keys input_size, hidden_size, output_size.
    
    :return final_string: str. 
    """
    hidden = initialize_hidden(opts['hidden_size'])
    loss = 0
    
    for i in range(len(input_seq)):
        x = input_seq[i]
        
        hidden = np.tanh(np.dot(params['U'], x) + np.dot(params['W'], hidden) + params['b_h'])
        output = softmax(np.dot(params['V'], hidden) + params['b_o'])

        loss += criterion(output, target_seq[i])
    
    return loss

In [14]:
loss_grad = grad(loss)

In [15]:
def create_one_hot(j, length):
    vec = np.zeros(length)
    vec[j] = 1
    return vec

In [16]:
def sample(params, initial, length, opts):
    """
    Sampling a string with a Recurrent neural network.
    
    :param params: dict of str: tensor, including keys U, W, v, b_h, b_o
    :param initial: str. Beginning character.
    :param length: length of the generated string.
    :param opts: dict of str: int. Including keys input_size, hidden_size, output_size.
    
    :return final_string: str. 
    """
    hidden = initialize_hidden(opts['hidden_size'])
    current_char = initial
    final_string = initial
    
    for i in range(length):
        x = create_one_hot(char_to_index[current_char], opts['input_size'])
        output, hidden = model(params, x, hidden)
        
        p = output
        current_index = np.random.choice(range(vocab_size), p=p.ravel())
        current_char = index_to_char[current_index]
        final_string += current_char
    
    return final_string

In [17]:
def main():
    # Use non-overlapping 25-character chunks for training
    sequence_length = 25
    num_epochs = 10
    print_every = 100
    evaluate_every = 100
    lr = 0.01

    opts = {
        'input_size': vocab_size,
        'hidden_size': 100,
        'output_size': vocab_size,
    }

    params = initialize_params(opts['input_size'], opts['hidden_size'], opts['output_size'])

    for ep in range(num_epochs):
        for i in range(data_length // sequence_length):
            start = i * sequence_length
            end = start + sequence_length + 1
            chunk = text[start:end]

            input_chars = chunk[:-1]
            target_chars = chunk[1:]

            input_seq = [char_to_index[c] for c in input_chars]
            target_seq = [char_to_index[c] for c in target_chars]

            input_seq_one_hot = [create_one_hot(j, vocab_size) for j in input_seq]

            example_loss = loss(params, input_seq_one_hot, target_seq, opts)

            grad_params = loss_grad(params, input_seq_one_hot, target_seq, opts)
            for param in params:
                gradient = np.clip(grad_params[param], -5, 5)
                
                '''
                Note that we are clipping gradients to prevent exploding gradients.
                As an experiment, try uncommenting the below line, and setting the
                learning rate to a higher value (>0.1) to observe the phenomena.
                '''
                # gradient = grad_params[param]
                
                params[param] -= lr * gradient

            if i % print_every == 0:
                print("LOSS = {}".format(example_loss))
                # print(grad_params)

            if i % evaluate_every == 0:
                sampled_string = sample(params, initial='a', length=100, opts=opts)
                print(sampled_string)

In [19]:
main()

## Part 2: Long Short-Term Memory Networks (LSTMs)

The LSTM improves upon several shortcomings of the RNN, in particular by allowing gradients to flow through long sequences without vanishing or exploding as fast as they would in RNNs. We will implement the LSTM, and then apply it again to the Shakespearian text dataset.

![Long Short-Term Memory Networks Diagram](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)
(Image from the [LSTM Tutorial](http://colah.github.io/posts/2015-08-Understanding-LSTMs/))

The update of an LSTM is given by the following equations:

$$
i_t = \sigma(U_i x_t + W_i h_{t-1} + b_i)
$$

$$
f_t = \sigma(U_f x_t + W_f h_{t-1} + b_f)
$$

$$
o_t = \sigma(U_o x_t + W_o h_{t-1} + b_o)
$$

$$
\tilde{C}_t = \tanh(U_C x_t + W_C h_{t-1} + b_C)
$$

$$
C_t = i_t * \tilde{C}_t + f_t * C_{t-1}
$$

$$
h_t = o_t * \tanh(C_t)
$$


In [20]:
def initialize_params(input_size, hidden_size, output_size):
    params = {
        'U_i': np.random.randn(hidden_size, input_size) * 0.01,
        'W_i': np.random.randn(hidden_size, hidden_size) * 0.01,
        'b_i': np.zeros(hidden_size),
        
        'U_f': np.random.randn(hidden_size, input_size) * 0.01,
        'W_f': np.random.randn(hidden_size, hidden_size) * 0.01,
        'b_f': np.zeros(hidden_size),
        
        'U_o': np.random.randn(hidden_size, input_size) * 0.01,
        'W_o': np.random.randn(hidden_size, hidden_size) * 0.01,
        'b_o': np.zeros(hidden_size),
        
        'U_c': np.random.randn(hidden_size, input_size) * 0.01,
        'W_c': np.random.randn(hidden_size, hidden_size) * 0.01,
        'b_c': np.zeros(hidden_size),
        
        'V': np.random.randn(output_size, hidden_size) * 0.01,
        'b': np.zeros(output_size)
    }
    return params

In [21]:
def sigmoid(x):
    return 1. / (1 + np.exp(-x))

def model(params, x, h_prev, C_prev):
    i_t = sigmoid(np.dot(params['U_i'], x) + np.dot(params['W_i'], h_prev) + params['b_i'])
    f_t = sigmoid(np.dot(params['U_f'], x) + np.dot(params['W_f'], h_prev) + params['b_f'])
    o_t = sigmoid(np.dot(params['U_o'], x) + np.dot(params['W_o'], h_prev) + params['b_o'])
    
    C_t_tilde = np.tanh(np.dot(params['U_c'], x) + np.dot(params['W_c'], h_prev) + params['b_c'])
    C_t = i_t * C_t_tilde + f_t * C_prev
    h_t = o_t * np.tanh(C_t)
    
    y = softmax(np.dot(params['V'], h_t) + params['b'])
    return y, h_t, C_t

In [22]:
def initialize_hidden(hidden_size):
    return np.zeros(hidden_size), np.zeros(hidden_size)

In [23]:
def loss(params, input_seq, target_seq, opts):
    """
    Compute the loss of RNN based on data.
    
    :param params: dict of str: tensor, including keys U, W, v, b_h, b_o.
    :param input_seq: list of str. Input string.
    :param target_seq: list of str. Target string.
    :param opts: dict of str: int. Including keys input_size, hidden_size, output_size.
    
    :return final_string: str. 
    """
    hidden, cell = initialize_hidden(opts['hidden_size'])
    loss = 0
    
    for i in range(len(input_seq)):
        x = input_seq[i]
        
        i_t = sigmoid(np.dot(params['U_i'], x) + np.dot(params['W_i'], hidden) + params['b_i'])
        f_t = sigmoid(np.dot(params['U_f'], x) + np.dot(params['W_f'], hidden) + params['b_f'])
        o_t = sigmoid(np.dot(params['U_o'], x) + np.dot(params['W_o'], hidden) + params['b_o'])

        C_t_tilde = np.tanh(np.dot(params['U_c'], x) + np.dot(params['W_c'], hidden) + params['b_c'])
        cell = i_t * C_t_tilde + f_t * cell
        hidden = o_t * np.tanh(cell)

        output = softmax(np.dot(params['V'], hidden) + params['b'])
        
        loss += criterion(output, target_seq[i])
    return loss

loss_grad = grad(loss)

In [24]:
def sample(params, initial, length, opts):
    """
    Sampling a string with a Recurrent neural network.
    
    :param params: dict of str: tensor, including keys U, W, v, b_h, b_o
    :param initial: str. Beginning character.
    :param length: length of the generated string.
    :param opts: dict of str: int. Including keys input_size, hidden_size, output_size.
    
    :return final_string: str. 
    """
    hidden, cell = initialize_hidden(opts['hidden_size'])
    current_char = initial
    final_string = initial
    
    for i in range(length):
        x = create_one_hot(char_to_index[current_char], opts['input_size'])
        output, hidden, cell = model(params, x, hidden, cell)
        
        p = output
        current_index = np.random.choice(range(vocab_size), p=p.ravel())
        current_char = index_to_char[current_index]
        final_string += current_char
    
    return final_string

In [26]:
main()


## Part 3: Batch Training w/ PyTorch Architecture Example 

In the next two sections, we will explore some methods to speed up RNN training. First, we will consider batch training on sequential data, in the case that data series are all of the same length.
The RNN cell is also reimplemented in PyTorch. 

Note:  
The below sampling technique should not be done in reality. This demo serves only to show how batch training is possible. Poor sampling probably contributes to why performance is slightly worse than online training.

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class RNN(nn.Module):
    ''' Identical to the RNN cell defined previously'''
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        self.tanh = nn.Tanh()
        
    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        hidden = self.tanh(hidden)
        output = self.h2o(hidden)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self, batch_size):
        return torch.zeros(batch_size, self.hidden_size)

In [28]:
def sample(rnn, initial, length):
    """
    Sampling a string with a Recurrent neural network.
    
    :param rnn: PyTorch RNN model
    :param initial: str. Beginning character.
    :param length: length of the generated string.
    
    :return final_string: str. 
    """
    
    current_char = initial
    final_string = initial
    
    hidden = rnn.initHidden(1).to(device)
    
    for i in range(length):
        x = create_one_hot(char_to_index[current_char], 65)
        
        # Create 1 char tensor
        x = torch.tensor(x).unsqueeze(0).float().to(device)
        output, hidden = rnn.forward(x, hidden)
        
        p = np.exp(output.cpu().detach().numpy())
        current_index = np.random.choice(range(vocab_size), p=p.ravel())
        current_char = index_to_char[current_index]
        final_string += current_char
    
    return final_string

In [29]:
def get_batch(text, seq_len, batch_size, vocab_size, idx):
    ''' 
    Sequentially defines non-overlapping batches defined by index.
    Ignores last bit of text. Don't do this in practice.
    A better sampling strategy would shuffle data. 
    This is not implemented for brevity.
    '''
    start = idx * seq_len * batch_size
    end = (idx + 1) * seq_len * batch_size + 1
    
    chunk = text[start:end]
    input_chars = chunk[:-1]
    target_chars = chunk[1:]
    
    input_seq = [char_to_index[c] for c in input_chars]
    target_seq = [char_to_index[c] for c in target_chars]

    input_seq_one_hot = [create_one_hot(j, vocab_size) for j in input_seq]
    
    # Convert chunks of text to batches of sequential text
    input_tensor = torch.tensor(input_seq_one_hot).reshape(-1, batch_size, vocab_size)
    target_tensor = torch.tensor(target_seq).reshape(-1, batch_size)
    
    # Send to GPU
    input_tensor = input_tensor.float().to(device)
    target_tensor = target_tensor.to(device)
    
    return input_tensor, target_tensor

In [31]:
n_hidden = 100
seq_len = 25
batch_size = 8

n_epochs = 5
lr = 0.001

rnn = RNN(vocab_size, n_hidden, vocab_size).to(device)

optimizer = optim.SGD(rnn.parameters(), lr=lr)
criterion = nn.NLLLoss()

for _ in range(n_epochs):
    for i in range(data_length // (batch_size * seq_len)):
        optimizer.zero_grad()
        
        # Get batches
        batch_input, batch_target = get_batch(text, seq_len, batch_size, vocab_size, i)

        # Initialize hidden state
        hidden = rnn.initHidden(batch_size).to(device)

        # Stores losses
        total_loss = 0
        for j in range(seq_len):
            # Get RNN output for batch
            output, hidden = rnn.forward(batch_input[j], hidden)
            # Calculate loss for batch
            loss = criterion(output, batch_target[j])
            total_loss += loss
        
        total_loss.backward()
        
        optimizer.step()
        
        if (i % 500 == 0):
            print("Training loss: {}\n".format(total_loss))
            s = sample(rnn, 'a', 100)
            print(s)

## Part 4: Batch Training with Irregular Sequences

One strength of the RNN is the ability to handle irregularly length sequences. However, this poses a challenge at train time. How do we iterate over the RNN with multiple sequences of different length? Two strategies exist:

- Online Training: Simply train one sequence at a time. Easy to implement, but slow
- Batch Training with Padding: Pad batch of sequences with zeros to match length of longest sequence in batch.

Online Training was shown in Part 1 / 2.  
Batch Training with Padding will be shown below using PyTorch's `pack_padded_sequence`


In [32]:
# First define generic RNN model
rnn = nn.RNN(1, 1)

In [33]:
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence

'''
The point of this exercise is to show padding, so we'll use a dummy batch of data.
Note irregular length of sequences. We use batches of dimension B x L x I where:
B = batch size, L = sequence length, I = input features
For this example, a batch has 3 samples with a max sequence length of 4, and 1 input feature.
'''
dummy_batch = [
    [[1], [3], [3], [4]],
    [[4], [1]],
    [[7]]
]

print("Original Input is:", dummy_batch, "\n")

'''
Notice that we can't easily input this to an RNN, or even represent it as a tensor.
We'll use padding to circumvent this.
'''

# Store original lengths of sequences before padding
orig_len = [len(x) for x in dummy_batch]

# Pad with zeros
max_len = len(max(dummy_batch, key=len))

for i in range(len(dummy_batch)):
    zero_pad = [[0]] * (max_len - len(dummy_batch[i]))
    dummy_batch[i] += zero_pad
    
print("Padded Input is:", dummy_batch, "\n")

# Convert to tensor and pack
dummy_tensor = torch.tensor(dummy_batch).float()
packed_tensor = pack_padded_sequence(dummy_tensor, orig_len, batch_first=True)

print(packed_tensor)

output, _ = rnn(packed_tensor)

# Apply the inverse transformation to recover the outputs
output = pad_packed_sequence(output)

# Notice that the output contains zeros where we added zero padding!
print(output[0])
# Reconstructed output also contains original indices
print(output[1])

# To recover the true output from where the input exists, just use the output
# from the range of the original input. We show an example for one sequence
true_output = []
for i in range(len(dummy_batch)):
    orig_len = output[1][i]
    true_output.append(output[0][orig_len-1, i])

print("Final output is:", true_output)

Original Input is: [[[1], [3], [3], [4]], [[4], [1]], [[7]]] 

Padded Input is: [[[1], [3], [3], [4]], [[4], [1], [0], [0]], [[7], [0], [0], [0]]] 

PackedSequence(data=tensor([[1.],
        [4.],
        [7.],
        [3.],
        [1.],
        [3.],
        [4.]]), batch_sizes=tensor([3, 2, 1, 1]), sorted_indices=None, unsorted_indices=None)
tensor([[[0.0819],
         [0.9541],
         [0.9987]],

        [[0.8601],
         [0.2624],
         [0.0000]],

        [[0.8949],
         [0.0000],
         [0.0000]],

        [[0.9674],
         [0.0000],
         [0.0000]]], grad_fn=<CopySlices>)
tensor([4, 2, 1])
Final output is: [tensor([0.9674], grad_fn=<SelectBackward>), tensor([0.2624], grad_fn=<SelectBackward>), tensor([0.9987], grad_fn=<SelectBackward>)]
